##  EDA Part 2

Now we will continue where we left off

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as pyoff

In [2]:
# Read df_revenue.csv into a DataFrame
df_revenue = pd.read_csv('df_revenue.csv')

# Read df.csv into a DataFrame
df = pd.read_csv('df.csv')

### Monthly Active Customers

Now we will look at the monthly active customers, the same way we processed Monthly revenue. For now we will look at UK customers only

In [69]:
#creating a new dataframe with UK customers only
df_uk = df.query("Country=='United Kingdom'").reset_index(drop=True)

#creating monthly active customers dataframe by counting unique Customer IDs
df_monthly_active = df_uk.groupby('YearMonth')['CustomerID'].nunique().reset_index()


#plotting the output
plot_data = [
    go.Bar(
        x=df_monthly_active['YearMonth'],
        y=df_monthly_active['CustomerID'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Active Customers'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

Now, does the trend we observe here extend to the number of orders as well ? let's check.

In [70]:

#create a new dataframe for no. of order by using quantity field
df_monthly_sales = df_uk.groupby('YearMonth')['Quantity'].sum().reset_index()

# #print the dataframe
# df_monthly_sales

#plot
plot_data = [
    go.Bar(
        x=df_monthly_sales['YearMonth'],
        y=df_monthly_sales['Quantity'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Total # of Order'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

Now let's go deeper and look at other important metrics, we will start with New Customer Ratio, but first what defines a new customer ?

We can assume a new customer is whoever did his/her first purchase in the time window we defined. We will do it monthly for this example.



### New Customer Ratio

Note: This cell contains a merge operationa and should be run only once.

In [71]:
#create a dataframe contaning CustomerID and first purchase date
df_min_purchase = df_uk.groupby('CustomerID').InvoiceDate.min().reset_index()
df_min_purchase.columns = ['CustomerID','MinPurchaseDate']
# Assuming df_min_purchase is your DataFrame
df_min_purchase['MinPurchaseDate'] = pd.to_datetime(df_min_purchase['MinPurchaseDate'], format='%Y-%m-%d %H:%M:%S')
df_min_purchase['MinPurchaseYearMonth'] = df_min_purchase['MinPurchaseDate'].dt.to_period('M').astype(str)


#merge first purchase date column to our main dataframe (df_uk)
df_uk = pd.merge(df_uk, df_min_purchase, on='CustomerID')

#if User's First Purchase Year Month before the selected Invoice Year Month
df_uk['UserType'] = 'New'
df_uk.loc[df_uk['YearMonth']>df_uk['MinPurchaseYearMonth'],'UserType'] = 'Existing'

#calculate the Revenue per month for each user type
df_user_type_revenue = df_uk.groupby(['YearMonth','UserType'])['Revenue'].sum().reset_index()

#filtering the dates and plot the result
df_user_type_revenue = df_user_type_revenue.query("YearMonth != '2010-12' and YearMonth != '2011-12'")
plot_data = [
    go.Scatter(
        x=df_user_type_revenue.query("UserType == 'Existing'")['YearMonth'],
        y=df_user_type_revenue.query("UserType == 'Existing'")['Revenue'],
        name = 'Existing'
    ),
    go.Scatter(
        x=df_user_type_revenue.query("UserType == 'New'")['YearMonth'],
        y=df_user_type_revenue.query("UserType == 'New'")['Revenue'],
        name = 'New'
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='New vs Existing'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)